# Lab 6 - review

## LPM vs Logistic Regression

### synthetic data 

Let's set up synthetic data representing students in the class. Assume ISchool students come to office hours more often. We want to predict the probability that a student comes to office hours, based on some observable characteristics.

In [ ]:
# let's set up test data

import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt 
%matplotlib inline

from IPython.display import display

n_students = 100
n_features = 5

p_ischool = 0.6
p_other = 0.3

odds_ischool = p_ischool/(1 - p_ischool)
odds_other = p_other/(1 - p_other)

log_odds_ischool = np.log(odds_ischool)
log_odds_other = np.log(odds_other)


print("ischool odds (3:2):", odds_ischool, "log-odds", log_odds_ischool)
print("non-ischool odds (3:7):", odds_other, "log-odds", log_odds_other)

np.random.seed(0)
students = pd.DataFrame(np.random.normal(size = (n_students, n_features)), columns=[f"feature_{i}" for i in range(1, n_features + 1)])
students["ischool"] = (students.index > (n_students/2)).astype(int) # set the first half of the dataframe to have ischool = 0
X = students.values
y = np.random.normal(loc = p_other, scale = 0.1, size = (n_students,)) + students["ischool"] * (p_ischool - p_other) + X.sum(axis = 1)/10
y = np.clip(y, a_min = 0, a_max = 1)
students["OH"] = y

plt.scatter(list(range(n_students//2)), y[:n_students//2], color = "crimson", label = "department: other")
plt.scatter(list(range(n_students//2, n_students)), y[n_students//2:], color = "dodgerblue", label = "department: ischool")
plt.legend(handlelength = 0.5, loc="upper center", ncols = 2)
plt.title("probability of attending office hours\n")
plt.xlabel("student id")
plt.ylabel("P(OH)")
plt.hlines([0, 1], 0, 100, color="gray", linestyle="dotted")
plt.ylim(-0.2, 1.2)
plt.xlim(-1, 101)
plt.show()


## LPM

The LPM is just a linear regression of probability on the student characteristics.

$$ y = X'\beta + \varepsilon $$

In [ ]:
# let's fit the model and see how well we did
X1 = sm.add_constant(X)
lpm = sm.OLS(y, X1).fit()
display(lpm.summary())

# now let's predict 
y_hat_lpm = lpm.predict(X1)

# zoinks
display(y_hat_lpm)
plt.scatter(list(range(n_students//2)), y_hat_lpm[:n_students//2], color = "crimson", label = "department: other")
plt.scatter(list(range(n_students//2, n_students)), y_hat_lpm[n_students//2:], color = "dodgerblue", label = "department: ischool")
plt.legend(handlelength = 0.5, loc="upper center", ncols = 2)
plt.title("predicted probability of attending office hours (LPM)\n")
plt.xlabel("student id")
plt.ylabel("P(OH)")
plt.hlines([0, 1], 0, 100, color="gray", linestyle="dotted")
plt.ylim(-0.2, 1.2)
plt.xlim(-1, 101)
plt.show()

## Take 2: Logistic Regression

In the LPM, there was nothing constraining predicted probabilities from staying in the valid range $[0, 1]$. To ensure predictions output valid probabilities, we can use a function that maps from the reals to the range $[0, 1]$, like the logistic.

In [ ]:
X_ = np.linspace(-10, 10)
m = 0.07
b = 0.5
Y = m*X_ + b

L = 1/(1 + np.exp(-X_))

plt.plot(X_, Y, color = "orange", label = "linear")
plt.plot(X_, L, color = "yellowgreen", label = "logit")
plt.legend()
plt.ylim(-0.5, 1.5)
plt.hlines([0, 1], X_[0], X_[-1], color="gray", linestyle="dotted")
plt.show()

### logistic regression transforms the data to constrain outputs to valid values:

$$ \log\left(\frac{y}{1-y}\right) \sim  X'\beta + \varepsilon $$

In [ ]:
# let's fit the model and see how well we did
X1 = sm.add_constant(X)
logreg = sm.Logit(y, X1).fit()
display(logreg.summary())

# now let's predict 
y_hat_logreg = logreg.predict(X1)

print(y_hat_logreg)
plt.scatter(list(range(n_students//2)), y_hat_logreg[:n_students//2], color = "crimson", label = "department: other")
plt.scatter(list(range(n_students//2, n_students)), y_hat_logreg[n_students//2:], color = "dodgerblue", label = "department: ischool")
plt.legend(handlelength = 0.5, loc="upper center", ncols = 2)
plt.title("predicted probability of attending office hours (logistic regression)\n")
plt.xlabel("student id")
plt.ylabel("P(OH)")
plt.hlines([0, 1], 0, 100, color="gray", linestyle="dotted")
plt.ylim(-0.2, 1.2)
plt.xlim(-1, 101)
plt.show()


logreg.params["x1"]

### Intepreting the logistic regression coefficients:

- start with the logistic function:
$$ \log\left(\frac{y}{1-y}\right) = X'\beta $$

- exponentiate:
$$ \frac{y}{1-y} = \exp\left(X'\beta\right) $$

- multiply by $(1-y)$
$$ y = (1- y)\exp\left(X'\beta\right) $$

- distribute 
$$ y = \exp\left(X'\beta\right) - y \exp\left(X'\beta\right) $$

- collect move terms with $y$ to the left
$$ y + y \exp\left(X'\beta\right) = \exp\left(X'\beta\right) $$

- factor out
$$ y(1 + \exp\left(X'\beta\right)) = \exp\left(X'\beta\right) $$

- divide
$$ y = \frac{\exp\left(X'\beta\right)}{1 + \exp\left(X'\beta\right)} $$

### just regress on ischool affiliation to work through calculation

In [ ]:
logreg_binary = sm.Logit(y, sm.add_constant(students["ischool"])).fit()

display(logreg_binary.summary())


a = logreg_binary.params["const"]
b = logreg_binary.params["ischool"]

def logistic(z):
    return np.exp(z)/(1 + np.exp(z))


# calculate log odds
print("estimated log odds for non-ischool", a, f"(actual: {log_odds_other})")

print("estimated log odds for ischool", a+b, f"(actual: {log_odds_ischool})")

# convert to probabilities
# non-ischool probability
z0 = a + b*0
p0 = logistic(z0)
print("logreg estimate of p(OH) (non-ischool): ", round(p0, 3), f"(actual: {p_other})")

# ischool probability 
z1 = a + b*1
p1 = logistic(z1)
print("logreg estimate of p(OH) (ischool): ", round(p1, 3), f"(actual: {p_ischool})") 



## things to remember:

1. LPMs can predict negative probability

2. Squashing your data through a function like the logistic function helps constrain your model output to valid values

3. Many functions map from $(-\infty, \infty) \mapsto [0, 1]$. We like the logistic because:
    
    a) the odds-ratio is somewhat interpretable
    
    b) the derivative is easy to calculate (we'll see this again in deep learning!)

4. Logistic regression coefficients interpreted as changes in log-odds (cf. [this guide to interpretation](https://stats.oarc.ucla.edu/other/mult-pkg/faq/general/faq-how-do-i-interpret-odds-ratios-in-logistic-regression/))